In [52]:
import numpy as np
import tensorflow as tf
import math
import os
from tensorflow.examples.tutorials.mnist import input_data as mnist_data

In [53]:
print ("Tensorflow version " + tf.__version__)

Tensorflow version 1.5.0


In [54]:
mnist = mnist_data.read_data_sets("data", one_hot=True)#, reshape=True, validation_size=0)

Extracting data/train-images-idx3-ubyte.gz
Extracting data/train-labels-idx1-ubyte.gz
Extracting data/t10k-images-idx3-ubyte.gz
Extracting data/t10k-labels-idx1-ubyte.gz


In [55]:
n_classes = 10
n_width   = 28
n_height  = 28
n_depth   = 1
n_inputs = n_height * n_width * n_depth # total pixels
learning_rate = 0.001
n_epochs = 10
batch_size = 100
n_batches = int(mnist.train.num_examples/batch_size)


In [56]:
print n_batches

550


In [57]:
X_train = mnist.train.images
X_test = mnist.test.images
Y_train = mnist.train.labels
Y_test = mnist.test.labels


from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets(os.path.join('.','mnist'), one_hot=True)
X_train = mnist.train.images
X_test = mnist.test.images
Y_train = mnist.train.labels
Y_test = mnist.test.labels

n_classes = 10 # 0-9 digits
n_width = 28
n_height = 28
n_depth = 1
n_inputs = n_height * n_width * n_depth # total pixels
learning_rate = 0.001
n_epochs = 10
batch_size = 100
n_batches = int(mnist.train.num_examples/batch_size)



In [58]:
print n_inputs, n_batches

784 550


In [59]:
X_ = tf.placeholder(dtype = tf.float32, shape=[None, n_inputs])
y = tf.placeholder(dtype = tf.float32,  shape=[None, 10])

In [60]:
X = tf.reshape(X_, shape = [-1, n_width, n_height, n_depth])

In [61]:
step = tf.placeholder(tf.int32)

In [62]:


# three convolution layers 
K = 4   # First layer depth
L = 8   # Second layer depth
M = 12  # Third layer depth
N = 200 # Fully connected 


W1 = tf.get_variable("W1", shape=[5,5,1,K], initializer=tf.contrib.layers.xavier_initializer())
B1 = tf.Variable(tf.ones([K])/10)
W2 = tf.get_variable("W2", shape=[5,5,K,L], initializer=tf.contrib.layers.xavier_initializer())
B2 = tf.Variable(tf.ones([L])/10)
W3 = tf.get_variable("W3", shape=[4,4,L,M], initializer=tf.contrib.layers.xavier_initializer())
B3 = tf.Variable(tf.ones([M])/10)
W4 = tf.get_variable("W4", shape=[7,7,M,N], initializer=tf.contrib.layers.xavier_initializer())
B4 = tf.Variable(tf.ones([N])/10)
W5 = tf.get_variable("W5", shape=[N,10], initializer=tf.contrib.layers.xavier_initializer())
B5 = tf.Variable(tf.ones([class_size])/10)

In [63]:
W1 = tf.Variable(tf.truncated_normal([5, 5, 1, K], stddev=0.1))  # 5x5 patch, 1 input channel, K output channels
B1 = tf.Variable(tf.ones([K])/10)
W2 = tf.Variable(tf.truncated_normal([5, 5, K, L], stddev=0.1))
B2 = tf.Variable(tf.ones([L])/10)
W3 = tf.Variable(tf.truncated_normal([4, 4, L, M], stddev=0.1))
B3 = tf.Variable(tf.ones([M])/10)

W4 = tf.Variable(tf.truncated_normal([7 * 7 * M, N], stddev=0.1))
B4 = tf.Variable(tf.ones([N])/10)
W5 = tf.Variable(tf.truncated_normal([N, 10], stddev=0.1))
B5 = tf.Variable(tf.ones([10])/10)

In [64]:
s = 1 # stride
y1 = tf.nn.relu(tf.nn.conv2d(X, W1, strides=[1,s,s,1], padding = 'SAME') + B1)

s = 2 # stride
y2 = tf.nn.relu(tf.nn.conv2d(y1, W2, strides=[1,s,s,1], padding = 'SAME') + B2)

s = 2 # stride
y3 = tf.nn.relu(tf.nn.conv2d(y2, W3, strides=[1,s,s,1], padding = 'SAME') + B3)

#falttern
yy = tf.reshape(y3, shape=[-1, 7 * 7 * M])
y4 = tf.nn.relu(tf.matmul(yy, W4) + B4)
yLogit = tf.matmul(y4, W5) + B5

prediction = tf.nn.softmax(yLogit)


In [71]:
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=yLogit, labels=y)
loss = tf.reduce_mean(cross_entropy) #* mini_batch_size

correct_prediction = tf.equal(tf.argmax(prediction,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))



In [66]:
lr = 0.0001 +  tf.train.exponential_decay(0.003, step, 2000, 1/math.e)
optimizer = tf.train.AdamOptimizer(lr).minimize(cross_entropy)

In [67]:
init = tf.global_variables_initializer()


In [73]:
sess = tf.Session()
sess.run(init)
for i in xrange(n_batches):
    batch_X, batch_y = mnist.train.next_batch(batch_size)
    a, c, _, _= sess.run([accuracy, loss, lr, optimizer ], feed_dict={X_ : batch_X, y : batch_y, step : i})
    #a += a
    #l += l
    print(str(i) + ": accuracy:" + str(a) + " loss: " + str(c) + " (lr:" + str(l) + ")")

#print('Train Accuracy is ', a/n_batches)
#print('Train Loss is ', l/n_batches)



TypeError: Fetch argument 0.0031 has invalid type <type 'numpy.float32'>, must be a string or Tensor. (Can not convert a float32 into a Tensor or Operation.)

In [ ]:
for i in xrange(100):
    batch_X, batch_y = mnist.test.next_batch(batch_size)
    a, l = sess.run([accuracy, loss],
                                feed_dict={X_ : batch_X , y : batch_y }) #.reshape(100, 28,28,1)
    a += a
    l += l
    print(str(i) + ": ********* epoch " + str(i*100//mnist.train.images.shape[0]+1) + " ********* test accuracy:" + str(a) + " test loss: " + str(c))
    
print('Test Accuracy is ', a/100)
print('Test Loss is ', l/100)


In [ ]:
sess.close()